In [ ]:
# All the necessary imports

import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain.tools import tool
from pydantic import BaseModel, Field
from langgraph.prebuilt import ToolNode
from langchain_tavily import TavilySearch

In [ ]:
load_dotenv(override=True)

In [ ]:
#Initializing State

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

In [ ]:
class GeneralSearch(BaseModel):
    """"Input for the general search tool"""
    query: str = Field(default="", description="Search query")

# class Chatbot_Output(BaseModel):
#   """Output Schema for the Chatbot"""
#   reply: str = Field(default="", description="Chatbot reply")
#   need_clarification: bool = Field(default=False, description="Need clarification. Only true when the chatbot needs clarification.")
#   search_tasks: list = Field(default=[], description="Search tasks for the tool")

In [ ]:
@tool(args_schema=GeneralSearch)
def general_search(query: str):
    """General search tool"""
    general_search = TavilySearch(
    max_results=5,
    topic="general"
)
    return general_search.invoke(query)

In [ ]:
system_message = """You are a medical information assistant. 
When a user asks a health question:
1. Use the general_search tool to find accurate medical information
2. After receiving search results, provide a clear, helpful answer
3. Include appropriate medical disclaimers when needed.
4. Always be sure to explain as if the user is in 5th grade. If some complex medical terms come up, be sure to simplify them. Don't use too much medical jargon.
Important: 
- Respond naturally in plain text, not JSON
- After you get tool results, synthesize them into a helpful answer
- Always inclue some relevant medical questions for your doctor about the query.
- For every query, when you give you verdict, always give a coloured alert. 
- Give a coloured alert to the user. 🟢 Green if the user is safe, 🟡 Yellow if the user is at risk, 🟠 Orange if the user is at high risk, 🔴 Red if the user is at very high risk.
- Following the colour alert, add an advise to the user.
After this continue with all the other important steps.
""".strip()

In [ ]:
# Initialize LLM with tools
llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools([general_search])

def chatbot_node(state: AgentState) -> AgentState:
    """Process messages and decide if tools are needed"""

    found_system_message = False
    messages = state["messages"]
    for message in messages:
        if isinstance(message, SystemMessage):
            message.content = system_message
            found_system_message = True

    if not found_system_message:
        messages = [SystemMessage(content=system_message)] + messages

    response = llm_with_tools.invoke(messages)
    
    return {
        "messages": [response],
    }
    
def should_continue(state: AgentState):
    """Check if we need to call tools"""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return "end"

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("chatbot", chatbot_node)
workflow.add_node("tools", ToolNode(tools=[general_search]))

workflow.add_edge(START, "chatbot")
workflow.add_conditional_edges(
    "chatbot",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)
workflow.add_edge("tools", "chatbot")

checkpointer = InMemorySaver()

graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
import gradio as gr
config = {"configurable": {"thread_id": "1"}}

def chat(message, history):
    try:
        # Extract user message content
        user_message = message if isinstance(message, str) else message.get("text", "")
        
        # Invoke the graph
        result = graph.invoke(
            {"messages": [HumanMessage(content=user_message)]}, 
            config=config
        )
        
        # Return the assistant's response in proper format
        assistant_response = result["messages"][-1].content
        return assistant_response
        
    except Exception as e:
        print(f"Chat error: {e}")
        return (f"Sorry, I encountered an error. -> {e}")

gr.ChatInterface(
    chat,
    title="Health Chatbot",
    description="Hi! What can I help you with?"
).launch()

In [ ]:
list(graph.get_state_history(config))

In [ ]:
"""
Health Chatbot Gradio Interface
Provides a web interface with voice input support for the health chatbot.
"""

import gradio as gr
from agent import get_chatbot_response

# Optional: Uncomment to enable audio transcription with OpenAI Whisper
from openai import OpenAI
client = OpenAI()


def transcribe_audio(audio_path):
    """
    Transcribe audio file to text using Whisper API.
    
    To enable this:
    1. pip install openai
    2. Uncomment the OpenAI import at the top
    3. Ensure OPENAI_API_KEY is in your .env file
    """
    # Uncomment the following to enable transcription:
    try:
        with open(audio_path, "rb") as audio_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        return transcript.text
    except Exception as e:
        print(f"Transcription error: {e}")
        return None
    
    # Placeholder - transcription not enabled
    return None


def chat(message, history):
    """
    Chat function that handles both text and audio inputs.
    
    Args:
        message: Either a string (text) or dict with 'text' and 'files' keys (multimodal)
        history: Chat history (managed by Gradio)
        
    Returns:
        The chatbot's response string
    """
    try:
        user_message = ""
        
        # Handle multimodal input (text + audio)
        if isinstance(message, dict):
            user_message = message.get("text", "")
            audio_files = message.get("files", [])
            
            # If audio is provided, try to transcribe it
            if audio_files:
                audio_path = audio_files[0]  # Get first audio file
                transcribed_text = transcribe_audio(audio_path)
                
                if transcribed_text:
                    # Use transcribed text if no text was provided
                    user_message = user_message or transcribed_text
                elif not user_message:
                    # No transcription available and no text provided
                    return ("🎤 **Audio transcription is not yet enabled.**\n\n"
                           "To enable voice input:\n"
                           "1. Install: `pip install openai`\n"
                           "2. Add your `OPENAI_API_KEY` to .env\n"
                           "3. Uncomment the transcription code in gradio_app.py\n\n"
                           "For now, please type your question.")
        else:
            # Plain text input
            user_message = message
        
        # Validate we have a message
        if not user_message or not user_message.strip():
            return "Please provide a message or voice input."
        
        # Get response from the agent
        response = get_chatbot_response(user_message)
        return response
        
    except Exception as e:
        print(f"Chat error: {e}")
        return f"Sorry, I encountered an error: {str(e)}"


# Create the Gradio interface
demo = gr.ChatInterface(
    fn=chat,
    title="🏥 Health Chatbot",
    description=(
        "Hi! I'm your health information assistant. Ask me about symptoms, "
        "conditions, or general health questions.\n\n"
        "💬 You can type your question or 🎤 use voice input (if enabled)."
    ),
    multimodal=True,  # Enables audio input button
    examples=[
        "What are the symptoms of flu?",
        "How can I improve my sleep quality?",
        "What should I do if I have a headache?",
    ],
)


if __name__ == "__main__":
    # Launch the interface
    demo.launch(
        share=False,  # Set to True to create a public link
        server_name="127.0.0.1",
        server_port=7860
    )
